In [6]:
USE_H5PY = False  # set to True to use h5py/hdf5lib instead
if USE_H5PY:
    import h5py
    from h5py import MultiManager
else:
    import h5pyd as h5py  # Use the "as" syntax for code  compatibility
    from h5pyd import MultiManager
import numpy as np

In [12]:
# create a new file
f = h5py.File("/home/test_user1/multi_try.h5", mode="w")

In [13]:
# create some datasets
DSET_SHAPE = (10,)
DSET_DTYPE = np.int32

# create 4 datasets
DSET_COUNT = 4
datasets = []
for i in range(DSET_COUNT):
    dset = f.create_dataset(f"dset_{i}", shape=DSET_SHAPE, dtype=DSET_DTYPE)
    datasets.append(dset)


In [18]:
# initialize some data to write
data_in = []
for n in range(DSET_COUNT):
    arr = np.zeros(DSET_SHAPE, dtype=DSET_DTYPE)
    arr[...] = list(range(n*100, n*100+DSET_SHAPE[0]))
    data_in.append(arr)


In [ ]:
# instantiate a MultiManager and use it to write to all the datasets simultaneously
mm = MultiManager(datasets)
mm[...] = data_in

In [19]:
# verify what get saved to the first dataset
dset = f["dset_0"]
dset[...]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [20]:
# and the second dataset
dset = f["dset_1"]
dset[...]

array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109], dtype=int32)

In [21]:
# Read all the data from all the daasets using the same MultiManager instance
data_out = mm[...]
len(data_out)

4

In [22]:
# get the first item from the returned list
data_out[0]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)

In [23]:
# and the second item
data_out[1]

array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109], dtype=int32)

In [27]:
# rather than reading all the data for a dataset, you can read a given selection
data_out = mm[0:4]
data_out[0]

array([0, 1, 2, 3], dtype=int32)

In [24]:
# it's also possible to pass a list of selections and have each selection
# read from the corresponding dataset
selections = []
for n in range(DSET_COUNT):
    s = slice(n, n+2, 1)
    selections.append(s)

data_out = mm.__getitem__(selections)

In [25]:
data_out[0]

array([0, 1], dtype=int32)

In [26]:
data_out[1]

array([101, 102], dtype=int32)